We rely on BeautifulSoup to scrap Futures Data related to cryptocurrencies from coinalyze.net.
The scraping code will be hosted and ran on deepnote each 15 minutes to gather data that will be used in ML and exploratory data analysis.

In [92]:
import requests
# from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd

In [93]:
web = requests.get("https://coinalyze.net/futures-data/coins/")
soup = bs(web.content, "html.parser")
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Cryptocurrency Futures Market Data: Open Interest, Funding Rate and Liquidations</title>
<meta content="Aggregated data for cryptocurrency futures market: open interest, funding rate, predicted funding rate, liquidations, statistics and more." name="description"/>
<meta content="" name="author"/>
<meta content="index" name="robots"/>
<meta content="Cryptocurrency Futures Market Data: Open Interest, Funding Rate and Liquidations" property="og:title"/>
<meta content="Aggregated data for cryptocurrency futures market: open interest, funding rate, predicted funding rate, liquidations, statistics and more." property="og:description"/>
<meta content="https://coinalyze.net/og-image.png" property="og:image"/>
<meta content="summary" name="twitter:card"/>
<link href="/favicon.png" rel="icon" type="image/png"/>
<link href="/css-bundle/mainTop.css?v=95" rel="stylesheet" type="text/css">
<script src="/api/assets-info/"></scrip

In [94]:
tablehead = soup.thead
tablehead

<thead>
<tr>
<th></th>
<th class="coin-name">Coin</th>
<th>
<div onclick="document.location='/futures-data/coins/?order_by=price&amp;order_dir=desc'">
<span title="Price">Price</span>
<span class="order"></span>
</div>
</th>
<th>
<div onclick="document.location='/futures-data/coins/?order_by=price_24hour_pchange&amp;order_dir=desc'">
<span title="Price Change 24 Hour">Change 24H</span>
<span class="order"></span>
</div>
</th>
<th>
<div onclick="document.location='/futures-data/coins/?order_by=volume_24hour&amp;order_dir=asc'">
<span title="Volume 24 Hour">Vol. 24H</span>
<span class="ordered-desc"></span>
</div>
</th>
<th>
<div onclick="document.location='/futures-data/coins/?order_by=oi_current&amp;order_dir=desc'">
<span title="Open Interest">Open Interest</span>
<span class="order"></span>
</div>
</th>
<th>
<div onclick="document.location='/futures-data/coins/?order_by=oi_24h_pchange&amp;order_dir=desc'">
<span title="Open Interest Change 24 Hour">OI Change 24H</span>
<span class="o

In [95]:
row_headers = []
for x in tablehead.find_all("tr"):
    for y in x.find_all("th"):
        row_headers.append(y.text.strip('\n'))
row_headers

['',
 'Coin',
 'Price',
 'Change 24H',
 'Vol. 24H',
 'Open Interest',
 'OI Change 24H',
 'OI Share',
 'OI / VOL24H',
 'Funding (avg)',
 'Pred. Funding (avg)',
 'Liqs. 24H']

In [96]:
tablebody = soup.body
tablebody

<body class="lang-en">
<script>window.REQ_KEY = '01e64f31-fe8e-48d5-9eeb-650562c3beef';</script>
<div class="body-wrapper">
<div class="nav-wrapper">
<div class="nav-desktop">
<div class="section-logo">
<a href="/"><img alt="Logo" src="/logo-symbol.png"/><span class="brand-name">Coinalyze</span></a>
</div>
<div class="sep"></div>
<div class="section-search">
<div>
<svg viewbox="0 0 12.27 12.83" xmlns="http://www.w3.org/2000/svg">
<defs><style>.cls-1,.cls-2{fill:none;stroke:#aaa;stroke-miterlimit:10;}.cls-2{stroke-linecap:round;}</style></defs>
<g data-name="Layer 2" id="Layer_2">
<g data-name="Layer 2" id="Layer_2-2"><circle class="cls-1" cx="4.5" cy="4.5" r="4"></circle><line class="cls-2" x1="7.12" x2="11.77" y1="7.68" y2="12.33"></line></g>
</g>
</svg>
</div>
<div class="symbol-search-global">
<input autocapitalize="off" autocomplete="off" placeholder="Search" type="text"/><ul class="symbol-search"></ul>
</div>
<script>
            document.addEventListener("jqueryReady", function()

In [97]:
tablevalues = []
for x in tablebody.find_all('tr')[1:]:
    td_tags = x.find_all('td')
    td_val = [y.text.strip('\n') for y in td_tags]
    tablevalues.append(td_val)
tablevalues

[['Open Interest Details\nFunding Rates Details\nLiquidations Details\nBasis Details',
  'Ethereum\nETH',
  '$1,539.24',
  '-2.04%',
  '$36.9b',
  '$6.9b',
  '-3.19%',
  '30.04%',
  '0.19',
  '-0.0032%',
  '-0.0101%',
  '$63.0m'],
 ['Open Interest Details\nFunding Rates Details\nLiquidations Details\nBasis Details',
  'Bitcoin\nBTC',
  '$19,834.19',
  '-1.75%',
  '$27.4b',
  '$9.7b',
  '-0.75%',
  '42.31%',
  '0.35',
  '+0.0014%',
  '-0.0039%',
  '$22.6m'],
 ['Open Interest Details\nFunding Rates Details\nLiquidations Details\nBasis Details',
  'Ethereum Classic\nETC',
  '$31.70',
  '-2.93%',
  '$1.5b',
  '$385.0m',
  '-0.87%',
  '1.68%',
  '0.26',
  '-0.0183%',
  '-0.0246%',
  '$3.3m'],
 ['Open Interest Details\nFunding Rates Details\nLiquidations Details\nBasis Details',
  'Solana\nSOL',
  '$30.94',
  '-3.57%',
  '$1.2b',
  '$383.5m',
  '-1.19%',
  '1.68%',
  '0.31',
  '+0.0060%',
  '+0.0051%',
  '$1.6m'],
 ['Open Interest Details\nFunding Rates Details\nLiquidations Details\nBasis D

In [98]:
df = pd.DataFrame(tablevalues, columns = row_headers)
df['Coin'] = df['Coin'].str.replace('\n', '-')
#Renaming first column for easier deletion
df =  df.rename(columns = {"": 'todelete'})
#Dropping "todelete" column
del df['todelete']

#Previewing our dataframe
df.head()



Coin       Price Change 24H Vol. 24H Open Interest  \
0          Ethereum-ETH   $1,539.24     -2.04%   $36.9b         $6.9b   
1           Bitcoin-BTC  $19,834.19     -1.75%   $27.4b         $9.7b   
2  Ethereum Classic-ETC      $31.70     -2.93%    $1.5b       $385.0m   
3            Solana-SOL      $30.94     -3.57%    $1.2b       $383.5m   
4            Tellor-TRB      $19.12    -16.58%    $1.2b        $67.4m   

  OI Change 24H OI Share OI / VOL24H Funding (avg) Pred. Funding (avg)  \
0        -3.19%   30.04%        0.19      -0.0032%            -0.0101%   
1        -0.75%   42.31%        0.35      +0.0014%            -0.0039%   
2        -0.87%    1.68%        0.26      -0.0183%            -0.0246%   
3        -1.19%    1.68%        0.31      +0.0060%            +0.0051%   
4       -16.40%    0.29%        0.06      -0.7172%            -0.8246%   

  Liqs. 24H  
0    $63.0m  
1    $22.6m  
2     $3.3m  
3     $1.6m  
4     $2.3m

In [105]:
from datetime import datetime
#Saving df to csv 
#Adding timestamp(date) to file name
df.to_csv('crypto_futures_data_{}.csv'.format(pd.datetime.now().strftime("%Y-%m-%d %H%M%S")))

C:\Users\15146\AppData\Local\Temp\ipykernel_9692\97770168.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df.to_csv('crypto_futures_data_{}.csv'.format(pd.datetime.now().strftime("%Y-%m-%d %H%M%S")))
